In [7]:
#!pip install tensorflow_hub
#!pip install tensorflow_text
#!pip install numpy
#!pip install tensorflow
#!pip install pandas
#!pip install SenticNet
#!pip install tensorflow.keras 

In [1]:
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np
import tensorflow as tf
import pandas as pd
# from senticnet.senticnet import SenticNet
import re
import string
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
#github test first commit

In [ ]:
# disable GPU
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
print(tf.__version__)

2.10.1


In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [ ]:
bert_preprocess_model = hub.KerasLayer(preprocess_url)
bert_model = hub.KerasLayer(encoder_url)

In [ ]:
# input : 126 words as sentence  if exceeds 126, split it, if less than 126 apply padding of 0.
# output : 128 * 768

def bert_process(text_to_process):
    text_preprocessed = bert_preprocess_model(text_to_process)

    inputk = {'input_word_ids' : [], 'input_mask': [], 'input_type_ids': []}
    
    for i in text_preprocessed.keys():
        index = 0

        for arr in text_preprocessed[i]:
            arr = list(arr.numpy())
            arr = list(filter(lambda num: num != 0, arr))
            if len(inputk[i]) == 0:
                inputk[i].append(arr)
            else:
                inputk[i][index] = list(filter(lambda num: num != 0, inputk[i][index]))
                if len(inputk[i][index]) + len(arr) -1 > 128:
                    inputk[i].append(arr)
                    lengthwithoutzero = len(inputk[i][index])
                    padd = [0]*(128-lengthwithoutzero)
                    inputk[i][index].extend(padd) 
                    index += 1
                else:
                    inputk[i][index].extend(arr[1:])             
        lengthwithoutzero = len(inputk[i][index])
        padd = [0]*(128-lengthwithoutzero)
        inputk[i][index].extend(padd)

    if len(inputk["input_word_ids"]) < 6:
        paddinput = [""] * ( 6-len(inputk["input_word_ids"]))
        padd2 = bert_preprocess_model(paddinput)
        for arr in padd2["input_word_ids"]:
            inputk["input_word_ids"].append(arr)
        for arr in padd2["input_mask"]:
            inputk["input_mask"].append(arr)

            
    inputk["input_mask"] = tf.convert_to_tensor(inputk["input_mask"])
    inputk["input_type_ids"] = tf.convert_to_tensor(inputk["input_type_ids"])
    inputk["input_word_ids"] = tf.convert_to_tensor(inputk["input_word_ids"])

    bert_results = bert_model(inputk)
    tempres = (bert_results["sequence_output"].numpy()).tolist()
    
    return tempres

In [ ]:
def clean_text(text):
    regex = re.compile('[%s]' % re.escape('|'))
    text = regex.sub(" ", text)
    words = str(text).split()
    words = [i.lower() + " " for i in words]
    words = [i for i in words if not "http" in i]
    words = " ".join(words)
    # words = words.translate(words.maketrans('', '', string.punctuation))
    return words

In [ ]:
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|watch)"
digits = "([0-9])"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "..." in text: text = text.replace("...","<prd><prd><prd>")
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("|||","<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [ ]:
MBTI = pd.read_csv('mbti_1.csv')
data = []

In [ ]:
# sample is number of samples being used.
# total data = 
# [sample, 6, 128, 768]
sample = 300

for n in range(1,int(sample/100) + 1):
    for post in MBTI["posts"][100*(n-1):(100 * n)]:
        sentence = clean_text(str(post))
        sentence = split_into_sentences(sentence)
        if len(sentence) < 50:
            temp = [""] * (50-len(sentence))
            sentence = sentence + temp
        processedinput = bert_process(sentence)
        data.append(processedinput[:6])

In [ ]:
print(len(data[0]))
print(len(processedinput[0]))
print(len(processedinput[0][0]))

6
128
768


In [ ]:
print(len(data))
for i in data:
    if len(i) != 6:
        print(len(i))

500


In [ ]:
'''
# labels = []
IElabels = []
NSlabels = []
FTlabels = []
JPlabels = []

templabels = MBTI["type"]
# labelmap = {"INTJ" : 0.0, "INTP": 1.0, "ENTJ": 2.0, "ENTP" : 3.0, "INFJ": 4.0, "INFP": 5.0, "ENFJ": 6.0, "ENFP": 7.0, "ISTJ": 8.0, "ISFJ": 9.0, "ESTJ": 10.0, "ESFJ": 11.0, "ISTP": 12.0, "ISFP": 13.0, "ESTP": 14.0, "ESFP": 15.0}
# labeindex = ["INTJ", "INTP", "ENTJ", "ENTP", "INFJ", "INFP", "ENFJ", "ENFP", "ISTJ", "ISFJ", "ESTJ", "ESFJ", "ISTP", "ISFP", "ESTP", "ESFP"]
labelmap = {"I" : 0.0, "E": 1.0, "N": 0.0, "S" : 1.0, "F": 0.0, "T": 1.0, "J": 0.0, "P": 1.0}

for l in templabels:
    IElabels.append(labelmap[l[0]])
    NSlabels.append(labelmap[l[1]])
    FTlabels.append(labelmap[l[2]])
    JPlabels.append(labelmap[l[3]])
'''

'\n# labels = []\nIElabels = []\nNSlabels = []\nFTlabels = []\nJPlabels = []\n\ntemplabels = MBTI["type"]\n# labelmap = {"INTJ" : 0.0, "INTP": 1.0, "ENTJ": 2.0, "ENTP" : 3.0, "INFJ": 4.0, "INFP": 5.0, "ENFJ": 6.0, "ENFP": 7.0, "ISTJ": 8.0, "ISFJ": 9.0, "ESTJ": 10.0, "ESFJ": 11.0, "ISTP": 12.0, "ISFP": 13.0, "ESTP": 14.0, "ESFP": 15.0}\n# labeindex = ["INTJ", "INTP", "ENTJ", "ENTP", "INFJ", "INFP", "ENFJ", "ENFP", "ISTJ", "ISFJ", "ESTJ", "ESFJ", "ISTP", "ISFP", "ESTP", "ESFP"]\nlabelmap = {"I" : 0.0, "E": 1.0, "N": 0.0, "S" : 1.0, "F": 0.0, "T": 1.0, "J": 0.0, "P": 1.0}\n\nfor l in templabels:\n    IElabels.append(labelmap[l[0]])\n    NSlabels.append(labelmap[l[1]])\n    FTlabels.append(labelmap[l[2]])\n    JPlabels.append(labelmap[l[3]])\n'

In [ ]:
labels = []
templabels = MBTI["type"]
labelmap = {"I" : 0.0, "E": 1.0, "N": 0.0, "S" : 1.0, "F": 0.0, "T": 1.0, "J": 0.0, "P": 1.0}
for l in templabels:
    labels.append([labelmap[l[0]], labelmap[l[1]], labelmap[l[2]], labelmap[l[3]]])

In [ ]:
print(templabels[1])

ENTP


#split train, val, test data


In [ ]:
print(labels[1])

[1.0, 0.0, 1.0, 1.0]


In [ ]:
Y_label = labels[:sample]

In [ ]:
'''
Y_IE_label = IElabels[:sample]
Y_NS_label = NSlabels[:sample]
Y_FT_label = FTlabels[:sample]
Y_JP_label = JPlabels[:sample]
'''

In [ ]:
# sum of val data and test data
TEST_SIZE = 0.4
# portion of val from test
VAL_SIZE = 0.5

In [ ]:
# split IE_label to test train
X_train, X_test, Y_train, Y_test = train_test_split(
    data, Y_label, test_size=TEST_SIZE, random_state=42)

In [ ]:
# split IE_label to test val
X_val, X_test, Y_val, Y_test = train_test_split(
    X_test, Y_test, test_size = VAL_SIZE, random_state=42)

In [ ]:
# split IE_label to test train
#X_train, X_IEtest, Y_train, Y_IEtest = train_test_split(
#    data, Y_IE_label, test_size=TEST_SIZE, random_state=42)

In [ ]:
# split IE_label to test val
#X_IE_val, X_IE_test, Y_IE_val, Y_IE_test = train_test_split(
#    X_IEtest, Y_IEtest, test_size = VAL_SIZE, random_state=42)

In [ ]:
# split NS_label to test train
#X_train, X_NStest, Y_train, Y_NStest = train_test_split(
#    data, Y_NS_label, test_size = TEST_SIZE, random_state=42)

In [ ]:
# split NS_label to test val
#X_NS_val, X_NS_test, Y_NS_val, Y_NS_test = train_test_split(
#    X_NStest, Y_NStest, test_size = VAL_SIZE, random_state=42)

In [ ]:
# split FT_label to test train
#X_train, X_FTtest, Y_train, Y_FTtest = train_test_split(
#    data, Y_FT_label, test_size = TEST_SIZE, random_state=42)

In [ ]:
# split FT_label to test val
#X_FT_val, X_FT_test, Y_FT_val, Y_FT_test = train_test_split(
#    X_FTtest, Y_FTtest, test_size = VAL_SIZE, random_state=42)

In [ ]:
# split JP_label to test train
#X_train, X_JPtest, Y_train, Y_JPtest = train_test_split(
#    data, Y_JP_label, test_size = TEST_SIZE, random_state=42)

In [ ]:
# split JP_label to test val
#X_JP_val, X_JP_test, Y_JP_val, Y_JP_test = train_test_split(
#    X_JPtest, Y_JPtest, test_size = VAL_SIZE, random_state=42)

# IE model

In [ ]:
#convert to tensor
X_train = tf.convert_to_tensor(X_train)
Y_train = tf.convert_to_tensor(Y_train)
X_val = tf.convert_to_tensor(X_val)
Y_val = tf.convert_to_tensor(Y_val)
X_test = tf.convert_to_tensor(X_test)
Y_test = tf.convert_to_tensor(Y_test)

In [ ]:
# model constructor for CNN classification
def model_fn() :
  model = models.Sequential()
  model.add(layers.Conv2D(32, (1, 3), activation='relu', input_shape=X_train.shape[1:]))
  model.add(layers.Conv2D(32, (3, 3), activation='relu'))
  model.add(layers.Conv2D(32, (3, 1), activation='relu'))
  
  # dropout
  # model.add(layers.Dropout(0.3))
  # fully connected layer
  model.add(layers.Flatten())
  model.add(layers.Dense(128, activation='sigmoid'))
  model.add(layers.Dense(4, activation='sigmoid'))
  #
  return model

In [ ]:
model = model_fn()
model.summary()

NameError: name 'model_fn' is not defined

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

checkpoint_cb = keras.callbacks.ModelCheckpoint('best-model.h5', 
                                                save_best_only=True)

early_stopping_cb = keras.callbacks.EarlyStopping(patience=3,
                                                 restore_best_weights=True)

history = model.fit(X_train, Y_train, epochs=20, batch_size = 32,
                    validation_data=(X_val, Y_val))
                    #callbacks=[checkpoint_cb, early_stopping_cb])


Epoch 1/100
10/10 [==============================] - 1s 67ms/step - loss: 0.5959 - accuracy: 0.1933 - val_loss: 0.5398 - val_accuracy: 0.2800
Epoch 2/100
10/10 [==============================] - 0s 45ms/step - loss: 0.5550 - accuracy: 0.2067 - val_loss: 0.5499 - val_accuracy: 0.2800
Epoch 3/100
10/10 [==============================] - 0s 48ms/step - loss: 0.5252 - accuracy: 0.2567 - val_loss: 0.5433 - val_accuracy: 0.2800
Epoch 4/100
10/10 [==============================] - 0s 45ms/step - loss: 0.4747 - accuracy: 0.3033 - val_loss: 0.5584 - val_accuracy: 0.3100
Epoch 5/100
 7/10 [====================>.........] - ETA: 0s - loss: 0.3671 - accuracy: 0.3929

: 

In [ ]:
model.evaluate(X_test, Y_test)

NameError: name 'IEmodel' is not defined

# Predict


In [ ]:
input = []
def preprocess_text(text) : 
    sentence = clean_text(str(text))
    sentence = split_into_sentences(sentence)
    if len(sentence) < 50:
        temp = [""] * (50-len(sentence))
        sentence = sentence + temp
    processedinput = bert_process(sentence)
    input.append(processedinput[:6])

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 8)

In [ ]:
input = tf.convert_to_tensor(input)
prediction = model.predict(input)
print(prediction)